In [144]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

from main import load_and_prepare_sessions
from processing.session_sampling import MiceAnalysis
from data.mouse import create_mice_dict
from plotting.main_plotting import plot_session_events_and_signal
from analysis.timepoint_analysis import sample_signals_and_metrics, find_drug_split_x
from itertools import product

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import scipy.stats as stats
from tqdm.notebook import tqdm
from collections import defaultdict

import matplotlib.pyplot as plt

import config

sessions = load_and_prepare_sessions("../../../trial_Gq-DREADD_CPT", load_from_pickle=True, remove_bad_signal_sessions=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [145]:
for session in sessions:
    for brain_region in session.brain_regions:
        threshold_x = find_drug_split_x(session, brain_region)
        brain_region = brain_region.split('_')[0]
        for event in config.all_event_types:
            curr_signal_info = session.signal_info.get((brain_region, event))
            if curr_signal_info is None:
                continue
            signal_idx_ranges = curr_signal_info['signal_idx_ranges']

            thresholds = np.array([((tup[0] + tup[1]) // 2) for tup in signal_idx_ranges])
            curr_signal_info["is_above_threshold"] = thresholds > threshold_x

In [146]:
def pair_wise_idxs(raw_df, actions_attr_dict):
    # Define events of interest including 'Display Image'
    events_of_interest = list(actions_attr_dict.keys())

    # Get raw data and filter for events of interest
    item_df = raw_df[["Item_Name"]]
    filtered_df = item_df[item_df["Item_Name"].isin(events_of_interest)].reset_index()

    # Initialize defaultdict to store the pairs
    event_pairs_dict = defaultdict(list)

    # Iterate through the filtered_df row by row
    for i in range(len(filtered_df) - 1):
        current_event = filtered_df.at[i, "Item_Name"]
        next_event = filtered_df.at[i + 1, "Item_Name"]
        if current_event in events_of_interest and next_event in events_of_interest:
            event_pairs_dict[(current_event, next_event)]\
                .append((filtered_df.at[i, "index"], filtered_df.at[i + 1, "index"]))

    return event_pairs_dict

In [147]:
def get_session_pair_signals(session, brain_region, dose, genotype, is_above=True):
    signal_order = []
    all_session_signals = []

    for event in (['hit', 'miss', 'mistake', 'cor_reject']):
        curr_signal_info = session.signal_info.get((brain_region, event))
        if curr_signal_info is None:
            continue
        curr_ranges = curr_signal_info['signal_idx_ranges']

        mask = curr_signal_info["is_above_threshold"]
        if not is_above:
            mask = ~mask

        len_before = sum(len(arr) for arr in all_session_signals)
        all_session_signals.append(curr_signal_info['signal_matrix'][mask])
        len_after = sum(len(arr) for arr in all_session_signals)

        event_list = [event] * len(curr_ranges)
        idx_list = range(len_before, len_after)
        signal_order.extend(zip(curr_ranges, event_list, idx_list))

    if len(signal_order) == 0:
        return None
    _, events, matrix_idx = zip(*sorted(signal_order, key=lambda x: x[0][0]))
    signal_matrix = np.vstack(all_session_signals)[matrix_idx, :]

    pair_dict = defaultdict(list)

    for i in range(len(events) - 1):
        curr_event = events[i]
        next_event = events[i + 1]
        #if next_event == 'hit':
        pair_dict[(curr_event, next_event, brain_region, dose, genotype, is_above)].append(signal_matrix[i+1])

    for k, v in pair_dict.items():
        pair_dict[k] = np.vstack(v)

    return pair_dict

In [148]:
pair_dict.keys()

dict_keys([('miss', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'hit', 'LH', 'Wildtype', None, True), ('hit', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'hit', 'LH', 'Wildtype', None, True), ('hit', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'mistake', 'LH', 'Wildtype', None, True), ('mistake', 'cor_reject', 'LH', 'Wildtype', None, True), ('mistake', 'mistake', 'LH', 'Wildtype', None, True), ('cor_reject', 'mistake', 'LH', 'Wildtype', None, True), ('hit', 'hit', 'LH', 'Wildtype', None, True), ('miss', 'cor_reject', 'mPFC', 'Wildtype', None, True), ('cor_reject', 'hit', 'mPFC', 'Wildtype', None, True), ('hit', 'miss', 'mPFC', 'Wildtype', None, True), ('miss', 'miss', 'mPFC', 'Wildtype', None, True), ('miss', 'hit', 'mPFC', 'Wildtype', None, True), ('hit', 'cor_reject', 'mPFC', 'Wildtype', None, T

In [149]:
pair_dict = defaultdict(list)

for is_above in (True, False):
    for session in sessions:
        for brain_region in config.all_brain_regions:
            for event in (['hit', 'miss', 'mistake', 'cor_reject']):
                if session.signal_info.get((brain_region, event)) is None:
                        continue

                genotype = session.genotype
                dose = session.drug_info['dose']

                pair_dict_part = get_session_pair_signals(session, brain_region, genotype, dose, is_above)
                if pair_dict_part is None:
                    continue
                for k, v in pair_dict_part.items():
                    pair_dict[k].append(v)

for k, v in pair_dict.items():
    pair_dict[k] = np.vstack(v)

In [150]:
for k, v in pair_dict.items():
    print(k, len(v), v.shape)

('miss', 'cor_reject', 'LH', 'Wildtype', None, True) 3248 (3248, 400)
('cor_reject', 'hit', 'LH', 'Wildtype', None, True) 5088 (5088, 400)
('hit', 'miss', 'LH', 'Wildtype', None, True) 1620 (1620, 400)
('miss', 'miss', 'LH', 'Wildtype', None, True) 1628 (1628, 400)
('miss', 'hit', 'LH', 'Wildtype', None, True) 1304 (1304, 400)
('hit', 'cor_reject', 'LH', 'Wildtype', None, True) 4488 (4488, 400)
('cor_reject', 'cor_reject', 'LH', 'Wildtype', None, True) 6212 (6212, 400)
('cor_reject', 'miss', 'LH', 'Wildtype', None, True) 3408 (3408, 400)
('miss', 'mistake', 'LH', 'Wildtype', None, True) 468 (468, 400)
('mistake', 'cor_reject', 'LH', 'Wildtype', None, True) 1412 (1412, 400)
('mistake', 'mistake', 'LH', 'Wildtype', None, True) 352 (352, 400)
('cor_reject', 'mistake', 'LH', 'Wildtype', None, True) 676 (676, 400)
('hit', 'hit', 'LH', 'Wildtype', None, True) 2356 (2356, 400)
('miss', 'cor_reject', 'mPFC', 'Wildtype', None, True) 4809 (4809, 400)
('cor_reject', 'hit', 'mPFC', 'Wildtype', Non

In [151]:
brain_reg_to_color = {'LH': 'orange',
                      'mPFC': 'cornflowerblue'}

In [152]:
def preprocess_and_plot_signals(pair_dict, event_type, brain_region, genotype, dose, is_above, smoothing_len=10):
    # Assuming all_signals is predefined
    event1, event2 = event_type
    signals = pair_dict[event1, event2, brain_region, genotype, dose, is_above]

    interval_start = config.peak_interval_config["interval_start"]
    interval_end = config.peak_interval_config["interval_end"]
    fps = config.PLOTTING_CONFIG['fps']
    
    xs = np.arange(-interval_start, interval_end) / fps
    
    # Smooth the mean signal
    ys = np.mean(signals, axis=0)
    window = np.ones(smoothing_len) / smoothing_len
    ys = np.convolve(ys, window, 'same')

    # Calculate the standard deviation of the mean
    std_signal = np.std(signals, axis=0) / np.sqrt(len(signals))

    # Use scipy.stats.norm.interval to get the 95% confidence interval
    alpha = 0.95
    ci_lower, ci_upper = stats.norm.interval(alpha, loc=ys, scale=std_signal)

    # The lower and upper bounds
    lb = ci_lower.min()
    ub = ci_upper.max()

    #ylim = (lb, ub)
    if brain_region == 'LH':
        plt.ylim(-.3, 0.5)
    else:
        plt.ylim(-0.25, 0.35)
    
    # Assuming brain_reg_to_color is predefined
    color = brain_reg_to_color[brain_region]

    plt.figure(dpi=300)
    plt.plot(xs[100: -100], ys[100: -100], color=color, label='Mean Signal')
    plt.fill_between(xs[100: -100], ci_lower[100: -100], ci_upper[100: -100], color=color, alpha=0.2, label='95% CI')
    #plt.ylim(ylim)
    plt.title(f'{event_type}, {brain_region}, {genotype}, {dose}, is above: {is_above}\n(n = {len(signals)})')
    plt.xlabel('Time (s)')
    plt.ylabel('z-score')
    plt.legend()
    plt.grid()
    plt.savefig(f'{event1}_to_{event2}_{brain_region}_{genotype}_{dose}_{is_above}.png')
    plt.show()

In [153]:
pair_dict.keys()

dict_keys([('miss', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'hit', 'LH', 'Wildtype', None, True), ('hit', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'hit', 'LH', 'Wildtype', None, True), ('hit', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'mistake', 'LH', 'Wildtype', None, True), ('mistake', 'cor_reject', 'LH', 'Wildtype', None, True), ('mistake', 'mistake', 'LH', 'Wildtype', None, True), ('cor_reject', 'mistake', 'LH', 'Wildtype', None, True), ('hit', 'hit', 'LH', 'Wildtype', None, True), ('miss', 'cor_reject', 'mPFC', 'Wildtype', None, True), ('cor_reject', 'hit', 'mPFC', 'Wildtype', None, True), ('hit', 'miss', 'mPFC', 'Wildtype', None, True), ('miss', 'miss', 'mPFC', 'Wildtype', None, True), ('miss', 'hit', 'mPFC', 'Wildtype', None, True), ('hit', 'cor_reject', 'mPFC', 'Wildtype', None, T

In [154]:
pair_dict.keys()

dict_keys([('miss', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'hit', 'LH', 'Wildtype', None, True), ('hit', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'hit', 'LH', 'Wildtype', None, True), ('hit', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'cor_reject', 'LH', 'Wildtype', None, True), ('cor_reject', 'miss', 'LH', 'Wildtype', None, True), ('miss', 'mistake', 'LH', 'Wildtype', None, True), ('mistake', 'cor_reject', 'LH', 'Wildtype', None, True), ('mistake', 'mistake', 'LH', 'Wildtype', None, True), ('cor_reject', 'mistake', 'LH', 'Wildtype', None, True), ('hit', 'hit', 'LH', 'Wildtype', None, True), ('miss', 'cor_reject', 'mPFC', 'Wildtype', None, True), ('cor_reject', 'hit', 'mPFC', 'Wildtype', None, True), ('hit', 'miss', 'mPFC', 'Wildtype', None, True), ('miss', 'miss', 'mPFC', 'Wildtype', None, True), ('miss', 'hit', 'mPFC', 'Wildtype', None, True), ('hit', 'cor_reject', 'mPFC', 'Wildtype', None, T

In [ ]:
for k in sorted([k for k in pair_dict.keys() if k is not None], key=lambda x: x[-3]):
    event1, event2, br, genotype, dose, is_above = k
    event = (event1, event2)
    preprocess_and_plot_signals(pair_dict, event, br, genotype, dose, is_above)